In [1]:
import numpy as np
from pymatgen.core import Structure
import pymatgen
from pymatgen.ext.matproj import MPRester
import pandas as pd
from pymatgen.core.composition import Composition
import matminer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

import matminer
from pymatgen.io.cif import CifParser
#from pymatgen.core.structure import Cifwriter
from pymatgen.core.structure import Structure
from matminer.datasets import load_dataset

In [2]:
api='D32oOhE5YEc2fubxEdCv1khjg6EBIfCL'
with MPRester(api) as mpr:
    search_results=mpr.summary.search(formula="ABC",fields=["structure","band_gap","composition"])

Retrieving SummaryDoc documents:   0%|          | 0/4684 [00:00<?, ?it/s]

In [3]:
    data_list=[]
    for result in search_results:
      data_dict={}
      data_dict['structure']=result.structure
      data_dict['composition']=result.composition.reduced_formula
      data_dict['bandgap']=result.band_gap
      data_list.append(data_dict)



In [4]:
#pd.set_option('display.max_columns',None)
cotst=pd.DataFrame(data_list)
cotst

,structure,composition,bandgap
0,"[[ 2.47450345 -1.42865846 8.81820568] Li, [2....",LiCaPb,0.0000
1,"[[3.15861007 3.3214221 5.02131638] Th, [0.507...",ThGeS,0.0000
2,"[[-2.60664608 -1.50490144 -4.66299835] Hf, [-2...",HfMnAl,0.0000
3,"[[ 1.999793 -1.98273754 1.505912 ] Tb, [1....",TbAlGe,0.0000
4,"[[1.04966183 2.16036022 4.18114524] Li, [1.167...",LiBeN,2.0847
...,...,...,...
4679,"[[5.57301876 2.40048659 1.07081013] Cu, [-0.42...",CuClO,0.2576
4680,"[[0. 0. 3.5390215] Er, [0. 0. 0....",ErZnGa,0.0000
4681,"[[0. 0. 0.] Ca, [3.345886 3.345886 3.345886] N...",CaNiTe,0.0000
4682,"[[1.11209789 6.99310124 2.23543604] Sm, [1.112...",SmGeRh,0.0000


In [5]:
cotst['composition']=cotst['composition'].apply(lambda x:Composition(x))

In [6]:
from matminer.featurizers.composition.orbital import AtomicOrbitals
Ato=AtomicOrbitals()
feat10=Ato.featurize_dataframe(cotst,col_id='composition',ignore_errors=True)

from matminer.featurizers.composition.element import BandCenter
band=BandCenter()
#band.fit(cotst['composition'])
feat13=band.featurize_dataframe(df=cotst,col_id='composition',ignore_errors=True)

from matminer.featurizers.composition.orbital import ValenceOrbital
val=ValenceOrbital()
feat7=val.featurize_dataframe(cotst,col_id='composition',ignore_errors=True)

AtomicOrbitals:   0%|          | 0/4684 [00:00<?, ?it/s]

BandCenter:   0%|          | 0/4684 [00:00<?, ?it/s]

ValenceOrbital:   0%|          | 0/4684 [00:00<?, ?it/s]

In [7]:
pd.set_option('display.max_columns',100)


In [8]:
feat7

,structure,composition,bandgap,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons
0,"[[ 2.47450345 -1.42865846 8.81820568] Li, [2....","(Li, Ca, Pb)",0.0000,1.666667,0.666667,3.333333,4.666667,0.161290,0.064516,0.322581,0.451613
1,"[[3.15861007 3.3214221 5.02131638] Th, [0.507...","(Th, Ge, S)",0.0000,2.000000,2.000000,4.000000,0.000000,0.250000,0.250000,0.500000,0.000000
2,"[[-2.60664608 -1.50490144 -4.66299835] Hf, [-2...","(Hf, Mn, Al)",0.0000,2.000000,0.333333,2.333333,4.666667,0.214286,0.035714,0.250000,0.500000
3,"[[ 1.999793 -1.98273754 1.505912 ] Tb, [1....","(Tb, Al, Ge)",0.0000,2.000000,1.000000,3.333333,3.000000,0.214286,0.107143,0.357143,0.321429
4,"[[1.04966183 2.16036022 4.18114524] Li, [1.167...","(Li, Be, N)",2.0847,1.666667,1.000000,0.000000,0.000000,0.625000,0.375000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
4679,"[[5.57301876 2.40048659 1.07081013] Cu, [-0.42...","(Cu, Cl, O)",0.2576,1.666667,3.000000,3.333333,0.000000,0.208333,0.375000,0.416667,0.000000
4680,"[[0. 0. 3.5390215] Er, [0. 0. 0....","(Er, Zn, Ga)",0.0000,2.000000,0.333333,6.666667,4.000000,0.153846,0.025641,0.512821,0.307692
4681,"[[0. 0. 0.] Ca, [3.345886 3.345886 3.345886] N...","(Ca, Ni, Te)",0.0000,2.000000,1.333333,6.000000,0.000000,0.214286,0.142857,0.642857,0.000000
4682,"[[1.11209789 6.99310124 2.23543604] Sm, [1.112...","(Sm, Ge, Rh)",0.0000,1.666667,0.666667,6.000000,2.000000,0.161290,0.064516,0.580645,0.193548


In [9]:
feat10=feat10.drop(['structure','composition','bandgap','HOMO_element','LUMO_element','HOMO_character','LUMO_character'],axis=1)
feat7=feat7.drop(['structure','composition','bandgap'],axis=1)
feat13=feat13.drop(['structure','composition','bandgap'],axis=1)


In [10]:
result=feat10.join(feat7)
result=result.join(feat13)

In [11]:
result

,HOMO_energy,LUMO_energy,gap_AO,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,band center
0,-0.141831,-0.141831,0.000000,1.666667,0.666667,3.333333,4.666667,0.161290,0.064516,0.322581,0.451613,3.298094
1,-0.172896,-0.172896,0.000000,2.000000,2.000000,4.000000,0.000000,0.250000,0.250000,0.500000,0.000000,NaN
2,-0.191136,-0.166465,0.024671,2.000000,0.333333,2.333333,4.666667,0.214286,0.035714,0.250000,0.500000,3.441508
3,-0.256311,-0.149882,0.106429,2.000000,1.000000,3.333333,3.000000,0.214286,0.107143,0.357143,0.321429,3.604141
4,-0.266297,-0.205744,0.060553,1.666667,1.000000,0.000000,0.000000,0.625000,0.375000,0.000000,0.000000,4.676924
...,...,...,...,...,...,...,...,...,...,...,...,...
4679,-0.202272,-0.202272,0.000000,1.666667,3.000000,3.333333,0.000000,0.208333,0.375000,0.416667,0.000000,6.541686
4680,-0.134905,-0.134905,0.000000,2.000000,0.333333,6.666667,4.000000,0.153846,0.025641,0.512821,0.307692,3.659631
4681,-0.226594,-0.210764,0.015830,2.000000,1.333333,6.000000,0.000000,0.214286,0.142857,0.642857,0.000000,4.201149
4682,-0.217760,-0.217760,0.000000,1.666667,0.666667,6.000000,2.000000,0.161290,0.064516,0.580645,0.193548,3.924912


In [12]:
Q = pd.DataFrame(columns=['mean_atomic_numbers', 'max_atomic_numbers', 'min_atomic_numbers', 'std_atomic_numbers',
                          'a_parameters', 'b_parameters', 'c_parameters', 'alpha_parameters', 'beta_parameters',
                          'gamma_parameters', 'mean_distance_matrix', 'max_distance_numbers', 'min_distance_numbers',
                          'std_distance_numbers'])
#y = pd.DataFrame(columns=['formation_energy_per_atom'])

def extract_features(structure):
    # Atomic number features
    atomic_numbers = [site.specie.number for site in structure.sites]
    mean_atomic_numbers = np.mean(atomic_numbers)
    max_atomic_numbers = np.max(atomic_numbers)
    min_atomic_numbers = np.min(atomic_numbers)
    std_atomic_numbers = np.std(atomic_numbers)

    # Lattice parameter features
    lattice = structure.lattice
    a_parameters = lattice.a
    b_parameters = lattice.b
    c_parameters = lattice.c
    alpha_parameters = lattice.alpha
    beta_parameters = lattice.beta
    gamma_parameters = lattice.gamma

    # Distance matrix features
    distance_matrix = structure.distance_matrix
    mean_distance_matrix = np.mean(distance_matrix)
    max_distance_numbers = np.max(distance_matrix)
    min_distance_numbers = np.min(distance_matrix)
    std_distance_numbers = np.std(distance_matrix)


    return {
        'mean_atomic_numbers': mean_atomic_numbers,
        'max_atomic_numbers': max_atomic_numbers,
        'min_atomic_numbers': min_atomic_numbers,
        'std_atomic_numbers': std_atomic_numbers,
        'a_parameters': a_parameters,
        'b_parameters': b_parameters,
        'c_parameters': c_parameters,
        'alpha_parameters': alpha_parameters,
        'beta_parameters': beta_parameters,
        'gamma_parameters': gamma_parameters,
        'mean_distance_matrix': mean_distance_matrix,
        'max_distance_numbers': max_distance_numbers,
        'min_distance_numbers': min_distance_numbers,
        'std_distance_numbers': std_distance_numbers
    }

for index, row in cotst.iterrows():
    #Extract the structure and formation energy for the current material
    structure = row['structure']
    bandgap= row['bandgap']

    #Extract the features from the structure
    features = extract_features(structure)

    #Add the features and formation energy to the X and y DataFrames
    Q = Q.append(features, ignore_index=True)
   # y = y.append({'formation_energy_per_atom': formation_energy}, ignore_index=True)

#pd.set_option('display.max_columns',None)

result=result.join(Q)

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

...

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
The frame.append method is deprecated and will be remove

In [13]:
result

,HOMO_energy,LUMO_energy,gap_AO,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,band center,mean_atomic_numbers,max_atomic_numbers,min_atomic_numbers,std_atomic_numbers,a_parameters,b_parameters,c_parameters,alpha_parameters,beta_parameters,gamma_parameters,mean_distance_matrix,max_distance_numbers,min_distance_numbers,std_distance_numbers
0,-0.141831,-0.141831,0.000000,1.666667,0.666667,3.333333,4.666667,0.161290,0.064516,0.322581,0.451613,3.298094,35.000000,82.0,3.0,33.950945,4.949008,4.949008,10.961761,90.000000,90.000000,120.000013,3.614011,6.180960,0.0,1.600509
1,-0.172896,-0.172896,0.000000,2.000000,2.000000,4.000000,0.000000,0.250000,0.250000,0.500000,0.000000,NaN,46.000000,90.0,16.0,31.790984,3.968304,3.968304,9.021735,102.704917,102.704919,90.000024,2.799661,4.732575,0.0,1.335161
2,-0.191136,-0.166465,0.024671,2.000000,0.333333,2.333333,4.666667,0.214286,0.035714,0.250000,0.500000,3.441508,36.666667,72.0,13.0,25.460209,5.235209,5.254078,8.403694,89.999934,89.999890,120.119474,3.054073,5.166295,0.0,1.215342
3,-0.256311,-0.149882,0.106429,2.000000,1.000000,3.333333,3.000000,0.214286,0.107143,0.357143,0.321429,3.604141,36.666667,65.0,13.0,21.483844,5.717484,5.717484,6.023648,90.000000,90.000000,139.053749,2.588216,3.873600,0.0,1.194680
4,-0.266297,-0.205744,0.060553,1.666667,1.000000,0.000000,0.000000,0.625000,0.375000,0.000000,0.000000,4.676924,4.666667,7.0,3.0,1.699673,4.508970,4.717035,4.862395,90.000000,103.813880,90.000000,2.182526,3.496603,0.0,0.829998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4679,-0.202272,-0.202272,0.000000,1.666667,3.000000,3.333333,0.000000,0.208333,0.375000,0.416667,0.000000,6.541686,18.000000,29.0,8.0,8.602325,6.246369,6.267229,5.389696,90.000000,114.133041,90.000000,2.633044,3.786256,0.0,0.980099
4680,-0.134905,-0.134905,0.000000,2.000000,0.333333,6.666667,4.000000,0.153846,0.025641,0.512821,0.307692,3.659631,43.000000,68.0,30.0,17.682383,4.407108,4.407108,7.078043,90.000000,90.000000,119.999994,2.734305,4.358768,0.0,1.307825
4681,-0.226594,-0.210764,0.015830,2.000000,1.333333,6.000000,0.000000,0.214286,0.142857,0.642857,0.000000,4.201149,33.333333,52.0,20.0,13.597385,4.731797,4.731797,4.731797,60.000000,60.000000,60.000000,2.031362,3.345886,0.0,1.446715
4682,-0.217760,-0.217760,0.000000,1.666667,0.666667,6.000000,2.000000,0.161290,0.064516,0.580645,0.193548,3.924912,46.333333,62.0,32.0,12.283684,4.448392,7.114197,7.392507,90.000000,90.000000,90.000000,3.179811,5.576147,0.0,1.243594


In [14]:
X=result
y=cotst['bandgap']

X = np.array(X)
y = np.array(y)

## Find rows that have at least one non-zero value
#mask = np.any(X != 0)
#X = X[mask]
#y = y[mask]


## Keep only rows that have at least one non-zero value


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with the mean here some array are entirely zero ,so mean impute do not work
imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Standardize the features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

models = [
    
    BayesianRidge(),
    RandomForestRegressor(),
    XGBRegressor(),
    GradientBoostingRegressor(),
    LGBMRegressor(),
    LinearRegression(),
    RandomForestRegressor(),
    SVR(),
    Lasso(alpha=0.1)
]

for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Model: {type(model).__name__}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R^2: {r2}")
    print("-" * 50)


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
feature_importances = result.importances_mean
sorted_idx = np.argsort(feature_importances)
feature_labels = ['HOMO_energy', 'LUMO_energy', 'gap_AO', 'avg s valence electrons', 'avg p valence electrons', 'avg d valence electrons', 'avg f valence electrons', 'frac s valence electrons',                 'frac p valence electrons', 'frac d valence electrons', 'frac f valence electrons',                 'band center', 'mean_atomic_numbers', 'max_atomic_numbers', 'min_atomic_numbers',                 'std_atomic_numbers', 'a_parameters', 'b_parameters', 'c_parameters', 'alpha_parameters',                'beta_parameters', 'gamma_parameters', 'mean_distance_matrix', 'max_distance_numbers',                 'min_distance_numbers', 'std_distance_numbers']

fig = plt.figure(figsize=(10, 10))
plt.yticks(fontsize=5)
plt.ylim([15, 15])
plt.barh(range(X.shape[1]), feature_importances[sorted_idx],height=0.3)
plt.yticks(range(X.shape[1]), np.array(feature_labels)[sorted_idx])
plt.xlabel("Feature Importance")
plt.title("Catbooster")
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

plt.scatter(y_test, y_pred)
plt.title("CatBoost Regressor")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')
plt.xlabel('Actual bandgap (eV)')
plt.ylabel('Predicted bandgap (eV)')
plt.show()

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R^2: {r2}")

